## Data Pre-processing

#### Import relevant modules

In [ ]:
#@title Run on TensorFlow 2.x
#%tensorflow_version 2.x
#from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#### Load the dataset

In [ ]:
df = pd.read_csv("weatherAUS.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(labels = ['Date','Location','Evaporation','Sunshine','Cloud3pm','Cloud9am','RISK_MM'],axis = 1,inplace = True)

In [ ]:
# Replace Nilai Yes / No
df['RainToday'].replace({'No':0,'Yes':1},inplace = True) 
df['RainTomorrow'].replace({'No':0,'Yes':1},inplace = True)

In [ ]:
#Remove missing values
df.dropna(inplace = True) 

In [ ]:
categorical = ['WindGustDir','WindDir9am','WindDir3pm']
df_final = pd.get_dummies(df,columns = categorical,drop_first=True)

In [ ]:
df_final.head()

## Separating Your Training and Testing Datasets

In [ ]:
X = df_final.drop(['RainTomorrow'],axis=1).values
y = df_final['RainTomorrow'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Transforming the Data

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Define the plotting function

In [ ]:
#@title Define the plotting function.

def plot_the_loss_curve(epochs, mse):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Mean Squared Error")

  plt.plot(epochs, mse, label="Loss")
  plt.legend()
  plt.ylim([mse.min()*0.95, mse.max() * 1.03])
  plt.show()  

print("Defined the plot_the_loss_curve function.")

## Artificial Neural Network

#### Define a deep neural net model

In [ ]:
def create_model(my_learning_rate):
    model = tf.keras.models.Sequential()
    
    # Define the first hidden layer with 6 nodes.
    model.add(tf.keras.layers.Dense(6, kernel_initializer = "uniform",activation = "relu", input_dim=58))
    # Define the second hidden layer with 1 nodes. sigmoid activation
    model.add(tf.keras.layers.Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])
    
#     model.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    
    return model

#### Define a training function

In [ ]:
def train_model(model, features_arr, label_arr, epochs, 
                batch_size=None):
    history = model.fit(features_arr, label_arr, batch_size, epochs )
    
    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch
    
    # To track the progression of training, gather a snapshot
    # of the model's mean squared error at each epoch. 
    hist = pd.DataFrame(history.history)
    mse = hist["mean_squared_error"]

    return epochs, mse

#### Call the functions to build and train a deep neural net

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 25
batch_size = 5000

# Establish the model's topography.
my_model = create_model(learning_rate)

# Train the model
epochs, mse = train_model(my_model, X_train, y_train, epochs, 
                          batch_size)
plot_the_loss_curve(epochs, mse)

print("\n Evaluate the new model against the test set:")
my_model.evaluate(x = X_test, y = y_test, batch_size=batch_size)

## Running Predictions on the Test Set

In [ ]:
y_pred = my_model.predict(X_test)
y_train_pred = my_model.predict_classes(X_train)

In [ ]:
print('Training Accuracy ---->',accuracy_score(y_train,y_train_pred))
print('Testing Accuracy  ---->',accuracy_score(y_test,y_pred.round()))